In [1]:
import os
os.chdir('/Users/ozgecanyuzer/Documents/GitHub/ludo_clembench')


In [2]:
from backends import ModelSpec, Model, get_model_for, load_model_registry
#need to add api key
#need to add model_name in the dict
#need to create key.json
load_model_registry()  # load default model registry from backends folder
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106") # works without registered entry when openai_api.py backend is available


In [90]:
lmm: Model = get_model_for(THIS_MODEL)

lmm.set_gen_args(temperature=0.0, max_tokens=400)


In [108]:
import re

# System prompt
with open('/Users/ozgecanyuzer/Documents/GitHub/ludo_clembench/games/ludo/resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    sys_prompt = f.read()

# Initial instruction
with open('/Users/ozgecanyuzer/Documents/GitHub/ludo_clembench/games/ludo/resources/initial_prompts/no_board_prompt_multiple_tokens.txt', 'r') as f:
    content = f.read()

prefix = "MY MOVE:"

class Game():
    def __init__(self, lmm, initial_msg, sys_prompt):
        self.num_fields = 23
        self.current_pos = {'X': 0, 'Y': 0}  # Track positions of both tokens
        self.msg = initial_msg  # initial message -> contains game instructions
        self.last_response = []
        self.sys_prompt = sys_prompt  # system prompt
        self.lmm = lmm  # language model object
        self.rolls = [6, 5, 4, 5, 6, 6, 2]  # list of rolls per turn
        self.board = "□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □"
        self.turn = 0  # turn counter
        self.context = []  # memorized context
        self.current_state = ''  # current state of the board
        self.prefix = "MY MOVE:"  # used to parse the response
        self.sign = " -> "
        self.tokens_out = {'X': False, 'Y': False}  # Track which tokens are on the board
        self.six_count = 0  # Counter for number of 6s rolled

    def make_move(self):
        while self.turn < len(self.rolls):
            if self.turn == 0:  # first turn prompt is slightly different
                in_m = f"Are you ready to play?\nBeginning State: {self.board}\n Turn num: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."
                self.add_message(self.msg + '\n' + in_m)
                init_prompt, init_resp, init_resp_text = self.lmm.generate_response(self.context)
                move = self.parse_reply(init_resp_text)
                if self.check_move(move, self.rolls[self.turn]) == True:
                    self.update_board(move)
                    self.add_message(init_resp_text, role='assistant')
                    self.turn += 1  # updates step
                    token, pos = move
                    self.tokens_out[token] = True
                    self.current_pos[token] = pos
                    self.last_response = init_resp_text
                    if self.rolls[self.turn - 1] == 6:
                        self.six_count += 1
                else:
                    print(f'fail at turn: {self.turn}')
                    print(f'roll: {self.rolls[self.turn]}')
                    print(f'{init_resp_text}')
                    self.last_response = init_resp_text
                    print(self.current_state)
            else:
                # if not first turn
                m = f"Current state: {self.current_state}\n Next turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\n Where will you move your token? Let's think step by step."
                self.add_message(m, role='user')
                given_prompt, response, response_text = self.lmm.generate_response(self.context)

                move = self.parse_reply(response_text)
                if self.check_move(move, self.rolls[self.turn]) == True:
                    self.update_board(move)
                    token, pos = move
                    self.current_pos[token] = pos
                    self.add_message(response_text, role='assistant')
                    self.turn += 1
                    if self.rolls[self.turn - 1] == 6:
                        self.six_count += 1
                    print(self.current_state)
                else:
                    print(f'fail at turn: {self.turn}')
                    print(f'roll: {self.rolls[self.turn]}')
                    print(f'{response_text}')
                    print(f'move: {move}')
                    print(self.current_state)

    def check_move(self, move, roll):
        token, new_pos = move
        if not self.tokens_out[token]:
            if roll == 6 and new_pos == 1:
                return True
            elif roll != 6 and new_pos == 0:
                return True
            else:
                return False

        if new_pos > self.num_fields:
            return False
        if self.current_pos[token] > new_pos:
            return False
        if roll + self.current_pos[token] == new_pos:
            return True
        else:
            return False

    def parse_reply(self, text):
        match = re.search(r'MY MOVE:\s*([XY])\s*->\s*(\d+)', text)
        if not match:
            raise ValueError("Invalid response format")
        token = match.group(1)
        position = int(match.group(2))
        return token, position

    def update_board(self, move):
        token, pos = move
        split_b = self.board.split()
        if self.tokens_out[token]:
            split_b[self.current_pos[token] - 1] = '□'
        split_b[pos - 1] = token
        self.board = ' '.join(split_b)
        self.current_state = self.board

    def add_message(self, message, role='user'):
        # add message to context
        if self.context == []:
            self.context = [
                {"role": "system", "content": self.sys_prompt}
            ]
        self.context.append({"role": role, "content": message})

    def reset(self):
        # resets game state
        self.turn = 0
        self.context = []


In [109]:
instance = Game(lmm, content, sys_prompt)

In [110]:
instance.turn


0

In [111]:
instance.make_move()


□ □ □ □ □ X □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □
Y □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □
fail at turn: 6
roll: 2
MY MOVE: Y -> 3

There are 22 empty fields and 2 occupied fields. In total: 22 + 2 = 24 fields. I have token Y on field 1 and token X on field 21. You have rolled 2. Since 1 + 2 = 3, I need to move my token to field number 3.
move: ('Y', 3)
Y □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □
2024-05-22 08:53:32,203 - backends.utils - WARNING - Found consecutive role assignments. These will be merged into one:
{'role': 'user', 'content': "Current state: Y □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □\n Next turn number: 6, Roll: 2.\n Where will you move your token? Let's think step by step."}
{'role': 'user', 'content': "Current state: Y □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □\n Next turn number: 6, Roll: 2.\n Where will you move your token?

KeyboardInterrupt: 

In [112]:
instance.context

[{'role': 'system',
  'content': 'You are an avid board game player who likes to play the game according to the given rules.\n\n'},
 {'role': 'user',
  'content': 'Welcome to ludogame! Your task is to move a from start to end across the board. I will give you a board with empty fields. An empty field is marked like this □.\nYour tokens is marked with X and Y. You tokens X or Y counts as an occupied field. The total number of fields remains unchanged throughout the game.\n\nGeneral instructions:\n1) Every turn I will give you the current state of the grid. You must tell me what your next move is.\n2) Your answers must contain the words "MY MOVE:", the instruction of how you want the token to move.\n3) You must keep track of the state of the board in order to win the game.\n4) Add a second token when the second 6 is rolled for the first time in the game. After that, decide on which token you want to move each round.\n5) You cannot have 2 tokens in the same field.\n6) You can only play on